# Predicting severe occurrences of automobile collisions in Seattle, WA

# ```Part 1: Pre-Processing```

## Importing Libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import missingno as msno
%matplotlib inline

*** 
## Download and Load collision dataset

### This dataset was downloaded on 22 September, 2020
!wget -O Collisions.csv 'http://data-seattlecitygis.opendata.arcgis.com/datasets/5b5c745e0f1f48e7a53acec63a0022ab_0.csv?outSR={%22latestWkid%22:2926,%22wkid%22:2926}'

In [ ]:
# Import collision dataset and display the first five rows
df = pd.read_csv("Collisions_Seattle.csv")
print(df.head())

### Review the characteristics of the data

In [ ]:
print(df.info())
print()
print(df.dtypes)

The dataset has 221,525 rows and 40 columns. This identifies multiple variables with a significant amount of missing data. For example: 

```INTKEY``` (71,936); ```EXCEPTRSNCODE``` (101,122); ```EXCEPTRSNDESC``` (11,779); ```INATTENTIONIND``` (30,188); ```PEDROWNOTGRNT``` (5,192); ```SPEEDING``` (9,929)

A large number of missing values could cause noise and bias in the results, therefore will need to be carefully evaluated. 

In [ ]:
df.describe()

This is a summary statisics of the variables - excluding those that are catagorical. 

This has given us a quick overview of the contents of the collision dataset.  
At this stage it would be benefitial to get a profile of each of the variables.  

***
## Determine which variables to exclude
### OBJECTID | INCKEY | COLDETKEY | LOCATION

```OBJECTID``` is an unique has a unique value for each incident, which corresponds with the dataframe index value, therefore, ```OBJECTID``` can be dropped.

```INCKEY```, ```COLDETKEY```, and REPORTNO each have unique values for each collision incident, therefore, they can be dropped.

```INTKEY``` has too many missing entries to be used.

```LOCATION``` is a catagorical field that can be substitued by latitude and longitude variables (```X``` and ```Y``` variables respectively).

### EXCEPTRSNCODE | EXCEPTRSNDESC

```EXCEPTRSNCODE``` and ```EXCEPTRSNDESC``` will be dropped since they have too many missing values to be used in a model.

In [ ]:
print (df['EXCEPTRSNCODE'].value_counts())
print ()
print (df['EXCEPTRSNDESC'].value_counts())

### SEVERITYCODE | SEVERITYDESC

```SEVERITYCODE``` and ```SEVERITYDESC``` are duplicate information. 

```SEVERITYCODE``` will be dropped since ```SEVERITYDESC``` has more information.

In [ ]:
# Determine the frequency of the values for 'INATTENTIONIND'
severity = pd.DataFrame()
severity['severity'] = df['SEVERITYDESC'].value_counts()
severity['percent'] = (severity['severity']/sum(severity['severity'])*100)
print ("Frequency of severity categories")
print ()
print (severity)

### INCDATE | INCDTTM

```INCDATE``` and ```INCDTTM``` both provide the date of the incident.  Only ```INCDTTM``` is required since it includes both the time and date.  

To conduct time series analysis, ```INCDTTM``` needs to be converted from a _string format to the pandas data-time format_.

In [ ]:
df['INCDTTM'] = df['INCDTTM'].apply(pd.to_datetime)
print (df['INCDTTM'][:10])

### INATTENTIONIND | PEDROWNOTGRNT

```INATTENTIONIND``` has a significant amount of missing data with only 30,188 values, which all have a value of 'Y' and pertian to the driver not paying attention while driving, and should be dropped.

```PEDROWNOTGRNT``` also has a significant amount of missing data with only 5,195 values, which all have a value of 'Y' and pertian to pedestrian right of way was not granted.

In [ ]:
#Determine the frequency of each value for 'INATTENTIONIND' and 'PEDROWNOTGRNT'
print ("Frequency of INATTENTIONIND values:", df['INATTENTIONIND'].value_counts())
print()
print ("Frequency of PEDROWNOTGRNT values: ", df['PEDROWNOTGRNT'].value_counts())

### SDOT_COLCODE | SDOT_COLDESC

```SDOT_COLCODE``` and ```SDOT_COLDESC``` correspond to the same information.   ```SDOT_COLCODE``` is the type of collision, and ```SDOT_COLDESC``` has the description of each type of collision. Therefore, we can drop ```SDOT_COLCODE``` since ```SDOT_COLDESC``` has more information.

In [ ]:
print ("Unique collision type for SDOT:        ",len(df['SDOT_COLCODE'].unique()))
print ("Unique collision description for SDOT: ",len(df['SDOT_COLDESC'].unique()))

### ST_COLCODE | ST_COLDESC

```ST_COLCODE``` is a code for a description of a collision.

```ST_COLDESC``` is the description of a collision based on the states collision coding dictionary.

Since these variables are duplicate information it was expected to have the same or similar unique values.  This was not the case.  The ```ST_COLCODE``` needed to be converted from a string to a numeric value.  Rerun of unique values then confirm that ```ST_COLCODE``` and ```ST_COLDESC``` have the same number of unique values.  Therefore, we can drop ```ST_COLCODE``` since ```ST_COLDESC``` has more information.

In [ ]:
print ("Unique collision type for SDOT:        ",len(df['ST_COLCODE'].unique()))
print ("Unique collision description for SDOT: ",len(df['ST_COLDESC'].unique()))

In [ ]:
df['ST_COLCODE'] = pd.to_numeric(df['ST_COLCODE'], errors='coerce')
print(df.head())
print (df.dtypes)

In [ ]:
print ("Unique collision type for SDOT:        ",len(df['ST_COLCODE'].unique()))
print ("Unique collision description for SDOT: ",len(df['ST_COLDESC'].unique()))

### SPEEDING

```SPEEDING``` has a significant amount of missing data with only 9929 values, which all have a value of 'Y' and pertian to whether or not speeding was a factor in the collision.

In [ ]:
#Determine the frequency of each value for 'SPEEDING'
print ("Frequency of SPEEDING values:", df['SPEEDING'].value_counts())


### SEGLANEKEY

```SEGLANEKEY``` contains 2101 unique values with the value '0' dominating with 218,489 of the observations. The distribution of unique values is highly skewed indicating that ```SEGLANEKEY``` should be dropped.

In [ ]:
print ("Unique values for SEGLANEKEY: ",len(df['SEGLANEKEY'].unique()))
print ()
print (df['SEGLANEKEY'].value_counts())

In [ ]:
#Distribution of SEGLANEKEY
plt.figure(figsize=(12,8))
sns.histplot(data=df, x='SEGLANEKEY', bins=50)
plt.title("Distribution of SEGLANEKEY")
plt.xlabel("Histogram with fixed size bins (bins=50)")
plt.show()

### CROSSWALKKEY

```CROSSWALKKEY``` contains 2343 unique values with the value '0' dominating with 217,283 of the observations. The distribution of unique values is highly skewed indicating that ```CROSSWALKKEY``` should be dropped.

In [ ]:
print ("Unique values for CROSSWALKKEY: ",len(df['CROSSWALKKEY'].unique()))
print ()
print (df['CROSSWALKKEY'].value_counts())

In [ ]:
#Distribution of SEGLANEKEY
plt.figure(figsize=(12,8))
sns.histplot(data=df, x='CROSSWALKKEY', bins=50)
plt.title("Distribution of CROSSWALKKEY")
plt.xlabel("Histogram with fixed size bins (bins=50)")

### Remove Fields

Remove the fields (columns) that have been identified as not required.

In [ ]:
df.drop(["OBJECTID", "STATUS", "INCKEY","COLDETKEY","REPORTNO","INTKEY","LOCATION", "EXCEPTRSNCODE","EXCEPTRSNDESC","SEVERITYCODE", "INCDATE","INATTENTIONIND","PEDROWNOTGRNT","SPEEDING", "SDOTCOLNUM","SDOT_COLCODE", "ST_COLCODE", "SEGLANEKEY", "CROSSWALKKEY"],axis=1,inplace=True)
df.head()

***
## Missing Values

### Identify variables with missing data

In [ ]:
df.info()

### Missing Values Matrix & Heat Map

Using the missingno library to plot and identify where the missing values are located in each column and correlations between missing values across different columns.

The white lines indicate missing values.  Most of the columns with missing values occur in common rows.  Note the strong correlation between COLLISIONTYPE, UNDERINF, WEATHER, ROADCOND, LIGHTCOND, and ST_COLDESC.

The percentage cutoff for this plot shows variables that are at most 99.9% complete.  Any remaining variables with missing data will be considered negligable.

In [ ]:
filtered_data = msno.nullity_filter(df, filter='bottom', p=0.999)
sorted_data = msno.nullity_sort(filtered_data, sort='descending')
msno.matrix(sorted_data.sample(221525))

### Missing Values Heat Map

The heat map is another view of the correlation of variables with missing data.  The strength of the correlation is shown with values in the heat map.

In [ ]:
msno.heatmap(df,filter='bottom', p=0.999)

The heat map confirms the correlation of the missing data for the same rows for ```COLLISIONTYPE```, ```UNDERINFL```, ```WEATHER```, ```ROADCOND```, ```LIGHTCOND```, and ```ST_COLDESC```.  Variable ```JUNCTIONTYPE``` has about a 30% correlation with these variables.

Another strong positive corrlation is with the ```X``` and ```Y``` variables (Latitude and Longitude respectively).  ```ADDRTYPE``` correlates with ```X``` and ```Y``` for about 70% of the rows.

### Drop Missing Values for Location Coordinates

Removing rows of data corresponding with missing data for ```X``` and ```Y``` should remove all missing data found in ```ADDRTYPE```.  This is verified with df.info() after removing the null values of ```X``` and ```Y```.

After dropping these null values there remains 214,050 rows.

***
## Investigate variables through visualization

In [ ]:
# Drop null values in X and Y
df = df.dropna(subset=['X','Y'])

# Verify all null values for ADDRTYPE are removed
df[['X','Y','ADDRTYPE']].info()

### Address Type

In [ ]:
plt.figure(figsize=(12,8))
splot = sns.countplot(data=df, x = 'ADDRTYPE', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Severity Description

In [ ]:
plt.figure(figsize=(12,8))
splot = sns.countplot(data=df, x = 'SEVERITYDESC', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Collision Type


In [ ]:
plt.figure(figsize=(12,8))
splot = sns.countplot(data=df, x = 'COLLISIONTYPE', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Junction Type

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=90,size=12)
splot = sns.countplot(data=df, x = 'JUNCTIONTYPE', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

###  Description of Collision

In [ ]:
plt.figure(figsize=(12,24))
splot = sns.countplot(data=df, y = 'SDOT_COLDESC', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_width(),), (p.get_width(), p.get_y() + p.get_height()/2.), ha = 'center', va = 'center', xytext = (15, 0), textcoords = 'offset points')

### Under the Influence

This is a binary variable that has two different binary designations. First is 'N' or 'Y'.  The second is '0' or '1'.  The '0' corresponds to 'N' and '1' is 'Y'.  Therefore, replace all '0' values with 'N' and '1' with 'Y'. 

In [ ]:
print (df['UNDERINFL'].value_counts())

In [ ]:
df['UNDERINFL'] = df['UNDERINFL'].replace({'N':0,'Y':1,'0':0,'1':1})

plt.figure(figsize=(12,8))
splot = sns.countplot(data=df, x = 'UNDERINFL', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Weather

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=90,size=12)
splot = sns.countplot(data=df, x = 'WEATHER', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Road Condition

In [ ]:
plt.figure(figsize=(12,8))
splot = sns.countplot(data=df, x = 'ROADCOND', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### Light Condition

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=90,size=12)
splot = sns.countplot(data=df, x = 'LIGHTCOND', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_height(), ), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

### State Collision Description

In [ ]:
plt.figure(figsize=(12,24))
splot = sns.countplot(data=df, y = 'ST_COLDESC', palette='ocean_d', alpha=0.75)

for p in splot.patches:
    splot.annotate(format(p.get_width(),), (p.get_width(), p.get_y() + p.get_height()/2.), ha = 'center', va = 'center', xytext = (15, 0), textcoords = 'offset points')

***
## Edit some variables values and formats
### Unknowns
At this stage all missing values are resolved except for those that fall into one of the variables catagory.  
These will be designated as the "Unknown" for each variable.

In [ ]:
df.loc[:,['COLLISIONTYPE','SDOT_COLDESC', 'JUNCTIONTYPE', 'UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','ST_COLDESC']] = df.loc[:,['COLLISIONTYPE','SDOT_COLDESC', 'JUNCTIONTYPE', 'UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','ST_COLDESC']].fillna('Unknown')

In [ ]:
print (df['COLLISIONTYPE'].value_counts())
print ()
print (df['SDOT_COLDESC'].value_counts())
print ()
print (df['JUNCTIONTYPE'].value_counts())
print ()
print (df['UNDERINFL'].value_counts())

In [ ]:
print (df['WEATHER'].value_counts())
print ()
print (df['ROADCOND'].value_counts())
print ()
print (df['LIGHTCOND'].value_counts())
print ()
print (df['ST_COLDESC'].value_counts())

### Date and Time
There will be many factors to analyze with regards to the date and time of events.  Therefore, the ```INCDTTM``` timestamp will be parsed into new fields.

In [ ]:
df['INCDTTM'].head()

In [ ]:
import datetime 
from datetime import date
# import calendar

df['year'] = df['INCDTTM'].apply(lambda x: x.year)
df['month'] = df['INCDTTM'].apply(lambda x: x.month)
df['day'] = df['INCDTTM'].apply(lambda x: x.day)
df['hour'] = df['INCDTTM'].apply(lambda x: x.hour)
df['minute'] = df['INCDTTM'].apply(lambda x: x.minute)
df['weekday'] = df['INCDTTM'].apply(lambda x: x.weekday())  # iso weekday (starts with Monday = 0)
# df['day_of_week'] = df['INCDTTM'].dt.day_name() # string name of weekday

df[['year','month','day','hour','minute','weekday']].head()

Approximately a forth of the time stamp (hours, minutes, seconds) have value of '0'.  Unless midnight is truly a bewitching hour these values should not be included when analyzing incidents by hour.

In [ ]:
temp = df[(df.hour > 0) & (df.minute > 0)]
plt.figure(figsize=(12,8))
plt.xticks(rotation=90,size=12)
splot = sns.countplot(data=temp, x = 'hour', palette='ocean_d', alpha=0.75)


In [ ]:
df.info()


In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.scatter(range(len(df['INJURIES'])),df['INJURIES'])
plt.subplot(1,2,2)
plt.scatter(range(len(df['SERIOUSINJURIES'])),df['SERIOUSINJURIES'])
plt.show()

In [ ]:
df = df[(df.INJURIES < 50) & (df.SERIOUSINJURIES < 30)]
plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.scatter(range(len(df['INJURIES'])),df['INJURIES'])
plt.subplot(1,2,2)
plt.scatter(range(len(df['SERIOUSINJURIES'])),df['SERIOUSINJURIES'])
plt.show()

In [ ]:
print(df.shape)
print()
print(df.head())

### Simplify the number of catagories per variable

#### Weather

In [ ]:
df['WEATHER'].value_counts()

In [ ]:
# combine similar catgories 
df['WEATHER'] = df['WEATHER'].replace('Other','Unknown')

# combine clear and partly cloudy conditions into single catagory
df['WEATHER'] = df['WEATHER'].replace('Clear','Clear or Partly Cloudy')
df['WEATHER'] = df['WEATHER'].replace('Partly Cloudy','Clear or Partly Cloudy')

# Combine severe weather conditions into single catagory
df['WEATHER'] = df['WEATHER'].replace('Fog/Smog/Smoke','Severe Conditions')
df['WEATHER'] = df['WEATHER'].replace('Sleet/Hail/Freezing Rain','Severe Conditions')
df['WEATHER'] = df['WEATHER'].replace('Blowing Sand/Dirt','Severe Conditions')
df['WEATHER'] = df['WEATHER'].replace('Blowing Snow','Severe Conditions')
df['WEATHER'] = df['WEATHER'].replace('Severe Crosswind','Severe Conditions')

df['WEATHER'].value_counts()

#### Road condition

In [ ]:
# combine similar catgories 
df['ROADCOND'] = df['ROADCOND'].replace('Other','Unknown')
df['ROADCOND'] = df['ROADCOND'].replace(['Snow/Slush','Ice'],'Snow/Ice')
df['ROADCOND'] = df['ROADCOND'].replace(['Standing Water','Oil'],'Wet')
df['ROADCOND'] = df['ROADCOND'].replace(['Sand/Mud/Dirt'],'Dry')
df['ROADCOND'].value_counts()

#### Light condition

In [ ]:
# combine similar catgories
df['LIGHTCOND'] = df['LIGHTCOND'].replace('Other','Unknown')
df['LIGHTCOND'] = df['LIGHTCOND'].replace(['Dark - No Street Lights','Dark - Street Lights Off'],'Dark - No Street Lights')
df['LIGHTCOND'] = df['LIGHTCOND'].replace(['Dusk','Dawn'],'Dusk/Dawn')
df['LIGHTCOND'].value_counts()

#### Severity

In [ ]:
# combine similar catgories
df['SEVERITYDESC'] = df['SEVERITYDESC'].replace(['Fatality Collision','Serious Injury Collision'],'Severe')
df['SEVERITYDESC'] = df['SEVERITYDESC'].replace(['Property Damage Only Collision','Injury Collision', 'Unknown'],'Not Severe')
df['SEVERITYDESC'].value_counts()

## Output Pre-Processing data

In [ ]:
df.to_csv('Collisions_clean.csv')

***
***
### End of Pre-Processing
***
***